In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
data_dir=cn.DATA_DIR
files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
paths = [os.path.join(data_dir, filename) for filename in files]

In [3]:
simple = SimpleSBML()
error_paths = []
for idx, path in enumerate(paths):
  try:
    simple.initialize(path)
    m = MESGraph(simple)
    m.analyze(simple.reactions, error_details=False)
  except:
    print("Index...", idx, path, "Showed Error")
    error_paths.append(path)
#  if simple.reactions:
#     m.analyze(simple.reactions, error_details=False)
#   else:
#     print("the following model doesn't have reactions")
#   print(files[idx] + "...")
#   print("Type I Error: ", m.type_one_error)
#   print("Type II Error: ", m.type_two_error)

Index... 75 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000596_url.xml Showed Error
Index... 238 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0568648427_url.xml Showed Error
Index... 243 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000410_url.xml Showed Error
Index... 289 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000081_url.xml Showed Error
Index... 317 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000075_url.xml Showed Error
Index... 442 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000094_url.xml Showed Error
Index... 538 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000353_url.xml Showed Error
Index... 687 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000627_url.xml Showed Error
Index... 692 /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0072364382_url.xml Showed Error


In [17]:
print(len(error_paths))
simple.initialize(error_paths[5])
for r in simple.reactions:
  print(r.makeIdentifier(is_include_kinetics=False))
  print(r.category)

9


NameError: name 'model_reference' is not defined

In [11]:
import tesbml
for error_path in error_paths:
  document = tesbml.readSBML(error_path)
  model = document.getModel()
  #pm.print_model(model)
  try:
    simple.initialize(error_path)
  except:
    print(error_path)
    print("showed error")

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000596_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0568648427_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000410_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000081_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000075_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000094_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000353_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000627_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0072364382_url.xml
showed error


In [7]:
#pm.print_model(model)
for rct in model.getReaction(0).getListOfReactants():
  print(rct.getSpecies())
  print("similar, ", rct.species)
  print(rct.stoichiometry)
  print("similar, ", rct.getStoichiometry())
print("rct done! now pdt")
for pdt in model.getReaction(0).getListOfProducts():
  print(pdt.getSpecies())
  print(pdt.stoichiometry)
  print("similar, ", pdt.getStoichiometry())
pm.print_model(model)

model.getReaction(3).getId()
[rct.stoichiometry for r in model.getReaction(3).getListOfReactants() \
if rct.species != "haha"]
#r = Reaction(model.getReaction(3))
#r.category

rct done! now pdt
s51
1.0
similar,  1.0
<Model MODEL1508180000 "Philipson2015 - Innate immune response modulated by NLRX1">
MyD88_a_HP:  -> s51;
MyD88_d: s51 -> ;
TRAF_d: s7 -> ;
NFkB_a1:  -> s4;
NFkB_d: s4 -> ;
CytoL_a:  -> s34;
CytoL_d: s34 -> ;
NLRX1_d: s49 -> ;
MyD88_a:  -> s51;
HP_a:  -> s18;
HP_CytoL: s18 -> ;
NLRX1_TF:  -> s49;
NFkB_a2:  -> s4;
RIG_a_HP:  -> RIG;
RIG_d: RIG -> ;
MAVS_a:  -> MAVS;
MAVS_d: MAVS -> ;
IRF_a1:  -> IRF;
IRF_a2:  -> IRF;
IRF_a3:  -> IRF;
IRF_d: IRF -> ;
IFN_a:  -> IFN;
IFN_d: IFN -> ;
NOD_a:  -> NOD1;
NOD_d: NOD1 -> ;
HP_IFN: s18 -> ;
TRAF_a:  -> s7;
TFa_a:  -> TFa;
TFa_d: TFa -> ;
TFi_a:  -> TFi;
TFi_d: TFi -> ;
CytoE_a:  -> CytoE;
CytoE_d: CytoE -> ;
X_a:  -> X;
X_d: X -> ;
HP_CytoE: s18 -> ;


[]

In [10]:
model.getLgetListOfReactants()

AttributeError: getListOfReactants

In [56]:
REACTION_1_1 = "reaction_1_1"
REACTION_n_1 = "reaction_n_1"
REACTION_1_n = "reaction_1_n"
REACTION_n_n = "reaction_n_n"
REACTION_BOUNDARY = "reaction_boundary"
ReactionCategory = collections.namedtuple('ReactionCategory',
    'category predicate')
REACTION_CATEGORIES = [
    ReactionCategory(category=REACTION_1_1,
        predicate=lambda x,y,z,w: (x==1) and (y==1) and (z==w)),
    ReactionCategory(category=REACTION_1_n,
        predicate=lambda x,y,z,w: ((x==1) and (y>1) and (z==1.00)) or ((x==1) and (y==1) and (z<w))),
    ReactionCategory(category=REACTION_n_1,
        predicate=lambda x,y,z,w: ((x>1) and (y==1) and (w==1.00)) or ((x==1) and (y==1) and (z>w))),
    ReactionCategory(category=REACTION_n_n,
        predicate=lambda x,y,z,w: ((x>1) and (y>1)) or ((x==1) and (y>1) and (z!=1.00)) or ((x>1) and (y==1) and (w!=1.00))),
    ReactionCategory(category=REACTION_BOUNDARY,
        predicate=lambda x,y,z,w: (x==0) or (y==0) or (z==0) or (w==0)),
    ]

In [57]:
for reaction in model.getListOfReactions():
  num_rct = len([r.species for r in reaction.getListOfReactants()])
  num_pdt = len([p.species for p in reaction.getListOfProducts()])
  stoi_rct = sum([r.stoichiometry for r in reaction.getListOfReactants()])
  stoi_pdt = sum([p.stoichiometry for p in reaction.getListOfProducts()])
  print("x =", num_rct, ";y =", num_pdt, ";z =", stoi_rct, ";w =", stoi_pdt)
  for reaction_category in REACTION_CATEGORIES:
    if reaction_category.predicate(num_rct, num_pdt, 
                                 stoi_rct, stoi_pdt):
      print(reaction_category.category)

x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
reaction_n_n
x = 3 ;y = 1 ;z = 4.0 ;w = 1.5
reaction_n_n
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1


In [49]:
pm.print_model(model)

<Model "Arnold2011_Damour2007_RuBisCO-CalvinCycle">
PGA_prod_Vc: RuBP + CO2 + NADPH -> PGA;
PGA_prod_Vo: RuBP + O2 + NADPH -> PGA;
PGA_cons: PGA -> RuBP;
NADPH_prod: NADP -> NADPH;


In [51]:
x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
((x>1) and (y==1) and (w!=1.00))

True

In [53]:
((x>1) and (y>1)) or ((x==1) and (y>1) and (z!=1.00)) or ((x>1) and (y==1) and (w!=1.00))

True